## Demo for reading generated zip files

Assumes zip file in /tmp/testdata.zip, and therein image and .mat file under /testdata. Otherwise fileformat like 300w-lp.

For creating such data, e.g. taking test/data/300wlp_miniset.zip as basis:

```
python expand_dataset.py --format 300wlp test/data/300wlp_miniset.zip /tmp/testdata
cd /tmp/
zip -r testdata.zip ./testdata/
```

In [29]:
import os
import numpy as np
from os.path import splitext
import zipfile
from pathlib import Path
import io
import scipy.io
import cv2
from typing import Any, List
from scipy.spatial.transform import Rotation
from collections.abc import Mapping
from ipywidgets import interact
from PIL import Image

from face3drotationaugmentation.common import AugmentedSample, FloatArray, UInt8Array
from face3drotationaugmentation.dataset300wlp import imdecode, parse_sample
import face3drotationaugmentation.vis as vis

In [33]:
class DatasetCustom300WLPLike(object):
    '''Reads from zipfiles structured like 300w-lp and aflw2000-3d.

    Adjust to your needs.
    '''
    def __init__(self, filename):
        self._zf = zf = zipfile.ZipFile(filename)
        self._matfiles = sorted(self._discover_samples(zf))

    @staticmethod
    def _discover_samples(zf):
        filenames = [
            f.filename
            for f in zf.filelist
            if splitext(f.filename)[1] == '.mat' and f.filename.startswith('testdata/')
        ]
        return filenames
    
    def __getitem__(self, i: int) -> tuple[str, AugmentedSample]:
        """Returns the sample and basename."""
        matfile = self._matfiles[i]

        with io.BytesIO(self._zf.read(matfile)) as f:
            data = scipy.io.loadmat(f)

        jpgbuffer = self._zf.read(splitext(matfile)[0] + '.jpg')
        img = imdecode(jpgbuffer)

        name = Path(matfile).stem

        sample = parse_sample(data, img)
        return name, sample

    def close(self):
        self._zf.close()

    def __len__(self):
        return len(self._matfiles)

    def __iter__(self):
        for i in range(len(self)):
            yield self[i]

In [34]:
ds = DatasetCustom300WLPLike('/tmp/testdata.zip')

In [35]:
@interact(i = (0, len(ds)-1))
def plot(i):
    _, sample = ds[i]
    img = sample.image.copy()
    vis.draw_pose(img, sample, 255, 2)
    display(Image.fromarray(img))

interactive(children=(IntSlider(value=18, description='i', max=36), Output()), _dom_classes=('widget-interact'…